In [5]:
!pip install sounddevice


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Fatma Abdullah\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [7]:
import streamlit as st
import sounddevice as sd
import numpy as np
import whisper
import requests
import pyttsx3
from reportlab.pdfgen import canvas
from io import BytesIO

ModuleNotFoundError: No module named 'sounddevice'

In [ ]:
# ========= إعدادات =========
OPENROUTER_KEY = "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"   # حطي المفتاح اللي جايبـاه من OpenRouter
OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"

In [ ]:
# تحميل موديل Whisper
@st.cache_resource
def load_whisper_model():
    return whisper.load_model("base")  # ممكن تغيري لـ "small" أو "medium" حسب الإمكانيات

model = load_whisper_model()

In [2]:
# ========= تسجيل الصوت =========
def record_audio(duration=5, fs=16000):
    st.info("📢 جاري التسجيل...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype="float32")
    sd.wait()
    st.success("✅ تم التسجيل")
    return np.squeeze(audio)

In [3]:
# ========= تحويل الصوت لنص =========
def transcribe_audio(audio, fs=16000):
    # Whisper محتاج ملف
    temp_file = "temp.wav"
    import soundfile as sf
    sf.write(temp_file, audio, fs)
    result = model.transcribe(temp_file, language="ar")
    return result["text"]


In [4]:
# ========= استدعاء OpenRouter =========
def chat_with_openrouter(prompt):
    headers = {
        "Authorization": f"Bearer {OPENROUTER_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "gpt-4o-mini",  # ممكن تغيري الموديل
        "messages": [
            {"role": "system", "content": "أنت مساعد ذكي بالعربية"},
            {"role": "user", "content": prompt}
        ]
    }
    r = requests.post(OPENROUTER_URL, headers=headers, json=payload)
    if r.status_code == 200:
        data = r.json()
        return data["choices"][0]["message"]["content"]
    else:
        return f"خطأ: {r.text}"


In [5]:
# ========= تحويل نص لصوت =========
def speak_text(text):
    engine = pyttsx3.init()
    engine.setProperty("rate", 160)   # سرعة
    engine.setProperty("voice", "arabic")  # حسب الأصوات المتاحة
    engine.say(text)
    engine.runAndWait()


In [6]:
# ========= حفظ الرد في ملف PDF =========
def save_pdf(text):
    buffer = BytesIO()
    c = canvas.Canvas(buffer)
    c.drawString(100, 800, "رد الشات بوت:")
    c.drawString(100, 780, text)
    c.save()
    buffer.seek(0)
    return buffer

In [ ]:
import streamlit as st
import speech_recognition as sr

st.title("🎙️ تحويل الكلام إلى نص")

if st.button("ابدأ التسجيل"):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        st.write("🎤 اتكلمي دلوقتي...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio, language="ar-AR")
            st.success(f"النص: {text}")
        except Exception as e:
            st.error(f"حصل خطأ: {e}")


In [7]:
# ========= واجهة Streamlit =========
st.title("🤖 شات بوت صوتي بالعربية (بايثون فقط)")

option = st.radio("اختيار الإدخال:", ["🎤 تسجيل صوت", "⌨️ كتابة نص"])

if option == "🎤 تسجيل صوت":
    duration = st.slider("مدة التسجيل بالثواني:", 3, 15, 5)
    if st.button("ابدأ التسجيل"):
        audio = record_audio(duration)
        text_input = transcribe_audio(audio)
        st.text_area("النص المستخرج:", text_input, height=100)
        if st.button("إرسال إلى الشات بوت"):
            reply = chat_with_openrouter(text_input)
            st.text_area("رد البوت:", reply, height=150)
            if st.button("تشغيل الرد صوتيًا"):
                speak_text(reply)
            if st.button("تحميل الرد PDF"):
                pdf_buf = save_pdf(reply)
                st.download_button("تحميل PDF", pdf_buf, "reply.pdf")

else:
    user_text = st.text_area("اكتب رسالتك هنا:", height=100)
    if st.button("إرسال إلى الشات بوت"):
        reply = chat_with_openrouter(user_text)
        st.text_area("رد البوت:", reply, height=150)
        if st.button("تشغيل الرد صوتيًا"):
            speak_text(reply)
        if st.button("تحميل الرد PDF"):
            pdf_buf = save_pdf(reply)
            st.download_button("تحميل PDF", pdf_buf, "reply.pdf")

2025-09-16 20:30:31.914 
  command:

    streamlit run c:\Users\Fatma Abdullah\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [ ]:
!streamlit run voice_chatbot1.ipynb


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Streamlit requires raw Python (.py) files, but the provided file has no extension.
For more information, please see https://docs.streamlit.io
